# Overview

- This notebook intends to walk through the steps of replicating Table 2.
- `df_combo` is the main sample as shown in Table 1.

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import wrds

from load_CRSP_fund import load_CRSP_combined_file
from load_mflink import load_mflink1

import config
OUTPUT_DIR = Path(config.OUTPUT_DIR)
path = Path(OUTPUT_DIR) / "main_sample.parquet"
df_combo = pd.read_parquet(path)

df_crsp = load_CRSP_combined_file()
df_mflink1 = load_mflink1()

# CRSP Mutual Fund Data

- CRSP data and mflink1 are merged based on `crsp_fundno`to obtain the appropriate `wficn`.
- Calculate `mret` and `mtna` for each `wficn`.
- The new CRSP data is then merged with `df_combo` by `year` and `wficn` to get the main sample's monthly returns. 

In [2]:
df_crsp = df_crsp.merge(df_mflink1, how="inner", on="crsp_fundno").reset_index(drop=True)

df_crsp = df_crsp.sort_values(["caldt", "wficn"])
df_crsp['mret'] = df_crsp['mret'].fillna(0)
df_crsp['lipper_class_name'] = df_crsp['lipper_class_name'].fillna('None')

df_crsp = df_crsp[~df_crsp['lipper_class_name'].astype(str).str.contains('International|Fixed Income|Precious Metal', case=False, regex=True)]
ret = df_crsp.groupby(["caldt", "wficn", 'lipper_class_name'])["mret"].mean().reset_index().rename(columns={"mret": "crsp_ret"})
tna = df_crsp.groupby(["caldt", "wficn", 'lipper_class_name'])["mtna"].sum().reset_index().rename(columns={"mtna": "crsp_tna"})
df_crsp = pd.merge(pd.merge(ret, tna, on=["caldt", "wficn", 'lipper_class_name'], how="inner"), 
                   df_crsp[["caldt", "wficn", 'lipper_class_name', 'index_fund_flag']], on=["caldt", "wficn", 'lipper_class_name'], how="inner").sort_values(["caldt", "wficn"])
df_crsp = df_crsp.drop_duplicates()
df_crsp = df_crsp.rename(columns={"caldt": "date"})

df_crsp['year'] = df_crsp['date'].dt.year.astype('int')
df_crsp = pd.merge(df_crsp, df_combo[['year', 'wficn']], on=["year", "wficn"], how="inner")

df_crsp['date'] = df_crsp['date'].dt.strftime('%Y%m').astype('int')
df_crsp

,date,wficn,lipper_class_name,crsp_ret,crsp_tna,index_fund_flag,year
0,198001,100010.0,None,0.121324,0.000,None,1980
1,198002,100010.0,None,0.026230,0.000,None,1980
2,198003,100010.0,None,-0.121406,59.671,None,1980
3,198004,100010.0,None,0.048182,0.000,None,1980
4,198005,100010.0,None,0.032958,0.000,None,1980
...,...,...,...,...,...,...,...
959178,202312,100870.0,Mid-Cap Core Funds,0.077495,119.600,None,2023
959179,202309,607465.0,Telecommunication Funds,-0.042763,6.300,D,2023
959180,202310,607465.0,Telecommunication Funds,-0.083162,5.700,D,2023
959181,202311,607465.0,Telecommunication Funds,0.049475,5.700,D,2023


# Fama French Factors

- Factor returns, `df_ff`, are pulled from Kenneth R. French's website.

In [3]:
df_ff = pd.read_csv(Path(config.DATA_DIR)/'manual'/'F-F_Research_Data_5_Factors_2x3.csv').drop(['RF'], axis=1)
df_mom = pd.read_csv(Path(config.DATA_DIR)/'manual'/'F-F_Momentum_Factor.csv')
df_ff = df_ff.merge(df_mom, how='inner', on=['date'])
df_ff = df_ff[(df_ff['date'] >= 198001) & (df_ff['date'] <= 201912)]
df_ff

,date,Mkt-RF,SMB,HML,RMW,CMA,MOM
198,198001,5.51,1.83,1.75,-1.70,1.64,7.55
199,198002,-1.22,-1.57,0.61,0.04,2.68,7.88
200,198003,-12.90,-6.93,-1.01,1.46,-1.19,-9.55
201,198004,3.97,1.05,1.06,-2.10,0.29,-0.43
202,198005,5.26,2.11,0.38,0.34,-0.31,-1.12
...,...,...,...,...,...,...,...
673,201908,-2.58,-3.24,-4.78,0.56,-0.68,6.92
674,201909,1.43,0.27,6.75,1.84,3.37,-6.81
675,201910,2.06,0.26,-1.91,0.44,-0.96,0.15
676,201911,3.87,0.44,-2.02,-1.59,-1.24,-2.66


- CRSP data and factor returns are merged, and for each fund i in month t, flow_{i,t} is calculated using the formula:

$$
\text{flow}_{i,t} = \frac{\text{TNA}_{i,t}}{\text{TNA}_{i,t-1}} \times (1 + \text{ret}_{i,t})
$$

In [4]:
df_reg = pd.merge(df_crsp[df_crsp['date'] <= 201912], df_ff, on=['date'], how="outer").sort_values(["date"])
flow = df_reg.groupby('wficn').apply(lambda d: d['crsp_tna']/(d['crsp_tna'].shift(1)) - (1+d['crsp_ret'])).reset_index().rename(columns={'level_1': 'index', 0: "flow"})
flow.set_index('index', inplace=True)
df_reg = pd.merge(df_reg, flow[['flow']], left_index=True, right_index=True).sort_values(['wficn', 'date'])
df_reg[['crsp_ret', 'flow']] *= 100
df_reg.replace([np.inf, -np.inf], np.nan, inplace=True)
df_reg= df_reg.fillna(0)
df_reg

,date,wficn,lipper_class_name,crsp_ret,crsp_tna,index_fund_flag,year,Mkt-RF,SMB,HML,RMW,CMA,MOM,flow
22936,199012,100001.0,None,2.796197,169.570,0,1990,2.46,0.67,-1.70,2.73,-1.89,0.29,0.000000
23026,199101,100001.0,None,2.169982,175.865,0,1991,4.69,3.91,-1.60,1.28,-3.99,-6.45,1.542349
23446,199102,100001.0,None,6.194690,188.858,0,1991,7.19,3.94,-0.58,-0.20,-0.33,-4.61,1.193363
23866,199103,100001.0,None,3.333333,200.123,0,1991,2.65,3.83,-1.39,-0.53,-1.06,2.87,2.631466
24286,199104,100001.0,None,0.161290,208.589,0,1991,-0.28,0.31,1.50,0.62,0.74,-2.36,4.069108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329402,200708,611013.0,S&P 500 Index Objective Funds,1.404213,34.300,D,2007,0.92,-0.37,-1.86,-1.21,-0.53,0.10,-0.817702
332001,200709,611013.0,S&P 500 Index Objective Funds,3.758655,35.200,D,2007,3.22,-2.44,-2.21,-0.52,-3.02,4.63,-1.134748
334600,200710,611013.0,S&P 500 Index Objective Funds,1.620591,35.200,D,2007,1.80,-0.08,-2.98,-0.30,-0.09,5.02,-1.620591
337199,200711,611013.0,S&P 500 Index Objective Funds,-4.221388,33.700,D,2007,-4.83,-3.02,-0.94,1.89,-0.29,0.99,-0.039975


# Reporting the mean, std, and percentiles of factor betas across all funds

- To replicate Panel A of Table 2, for each fund i in month t, we run the following rolling time-series regression:
$$
\text{ret}_{i,t+1-k} = \alpha_{i,t} + \beta_{\text{MKT} i,t} \times \text{MKT}_{t+1-k} + \beta_{\text{HML} i,t} \times \text{HML}_{t+1-k} + \beta_{\text{SMB} i,t} \times \text{SMB}_{t+1-k} + \beta_{\text{MOM} i,t} \times \text{MOM}_{t+1-k} + \beta_{\text{CMA} i,t} \times \text{CMA}_{t+1-k} + \beta_{\text{RMW} i,t} \times \text{RMW}_{t+1-k} + \beta_{\text{flow} i,t} \times \text{flow}_{i,t+1-k} + \epsilon_{i,t,t+1-k}
$$
where k = 1,2,...,60.

- We require a fund should have 60 months of returns data and each rolling window contains 24 monthly observationswe need to run regression

In [5]:
from sklearn.linear_model import LinearRegression

def regression(df):
    beta = pd.DataFrame(columns = ['Mkt-RF', 'SMB', 'HML', 'MOM', 'CMA', 'RMW', 'flow'])

    for fund, data in df.groupby('wficn'):
        if len(data) >= 60: 
            for month in range(len(data)-59):
                sample = data.iloc[month:month+60, :]
                for rw in range(len(sample)-23):
                    rolling_window = sample.iloc[rw:rw+24, :]
                    X = rolling_window[['Mkt-RF', 'SMB', 'HML', 'MOM', 'CMA', 'RMW', 'flow']]
                    y = rolling_window[['crsp_ret']]
                    model = LinearRegression().fit(X, y)
                    coef = pd.DataFrame((model.coef_).reshape(-1, 7), columns = ['Mkt-RF', 'SMB', 'HML', 'MOM', 'CMA', 'RMW', 'flow'])
                    beta = pd.concat([beta, coef], axis=0)
    return beta

In [11]:
#df_reg = df_reg.sample(frac=0.3, random_state=42)
all_funds = regression(df_reg)
panelA = all_funds.describe().loc[['mean', 'std']].append(all_funds.quantile(0.05)).append(all_funds.describe().loc[['25%', '50%', '75%']]).append(all_funds.quantile(0.95))
panelA = panelA.rename(index={0.05: 'P5', '25%': 'P25', '50%': 'P50', '75%': 'P75', 0.95: 'P95'})
panelA

C:\Users\Chen Keyi\AppData\Local\Temp\ipykernel_7632\1135329579.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  panelA = all_funds.describe().loc[['mean', 'std']].append(all_funds.quantile(0.05)).append(all_funds.describe().loc[['25%', '50%', '75%']]).append(all_funds.quantile(0.95))
C:\Users\Chen Keyi\AppData\Local\Temp\ipykernel_7632\1135329579.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  panelA = all_funds.describe().loc[['mean', 'std']].append(all_funds.quantile(0.05)).append(all_funds.describe().loc[['25%', '50%', '75%']]).append(all_funds.quantile(0.95))
C:\Users\Chen Keyi\AppData\Local\Temp\ipykernel_7632\1135329579.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  panelA = all_funds.describe().loc[['mea

,Mkt-RF,SMB,HML,MOM,CMA,RMW,flow
mean,0.983693,0.218380,0.015560,0.014791,-0.033623,0.034930,0.016351
std,0.195336,0.374762,0.365291,0.181072,0.421511,0.361378,0.169359
P5,0.670947,-0.254075,-0.569421,-0.261567,-0.699967,-0.567813,-0.197304
P25,0.897052,-0.067077,-0.185752,-0.081127,-0.255027,-0.152983,-0.020881
P50,0.991317,0.129329,0.019062,0.005942,-0.018797,0.055047,0.000883
P75,1.084299,0.475689,0.217634,0.107632,0.193378,0.237557,0.049150
P95,1.268732,0.907977,0.572267,0.313200,0.609087,0.571854,0.277089


# Reporting the mean factor betas by Lipper mutual fund classifications

- To replicate Panel B of Table 2, we classify funds according to `lipper_class_name`, and then run the regressions again. 

In [7]:
df_growth = df_reg[df_reg['lipper_class_name'].astype(str).str.contains('Growth', case=False, regex=True)]
df_value = df_reg[df_reg['lipper_class_name'].astype(str).str.contains('Value', case=False, regex=True)]
df_base = df_reg[df_reg['lipper_class_name'].astype(str).str.contains('Base', case=False, regex=True)]
df_large_cap = df_reg[df_reg['lipper_class_name'].astype(str).str.contains('Large-Cap', case=False, regex=True)]
df_mid_cap = df_reg[df_reg['lipper_class_name'].astype(str).str.contains('Mid-Cap', case=False, regex=True)]
df_small_cap = df_reg[df_reg['lipper_class_name'].astype(str).str.contains('Small-Cap', case=False, regex=True)]

In [8]:
growth = regression(df_growth)
value = regression(df_value)
base = regression(df_base)
large_cap = regression(df_large_cap)
mid_cap = regression(df_mid_cap)
small_cap = regression(df_small_cap)
panelB = pd.DataFrame({'All': all_funds.mean(), 'Growth': growth.mean(), 'Value': value.mean(), 
              'Large cap': large_cap.mean(), 'Medium cap': mid_cap.mean(), 'Small cap': small_cap.mean()}).T
panelB

,Mkt-RF,SMB,HML,MOM,CMA,RMW,flow
Growth,1.038637,0.291185,-0.196650,0.079443,-0.221629,-0.143442,0.016727
Value,0.991588,0.183930,0.252841,-0.062765,0.142530,0.141713,0.007775
Large cap,0.995346,-0.078187,-0.041061,0.010867,-0.054420,0.018725,-0.011855
Medium cap,1.046651,0.310284,-0.010113,0.024348,-0.017866,0.036239,0.022407
Small cap,0.986165,0.720407,0.117639,0.024993,-0.082650,0.056936,0.012278


# Reporting the mean factor betas by index fund status
- To replicate Panel C of Table 2, we classify funds according to index fund status.
- `index_fund_flag` identifies if a fund is an index fund:
- B = index-based fund
- D = pure index fund
- E = index fund enhanced

In [9]:
df_index = df_reg[df_reg['index_fund_flag'].astype(str).str.contains('D|B|E', case=False, regex=True)]
df_enhanced = df_reg[df_reg['index_fund_flag'].astype(str).str.contains('E', case=False, regex=True)]
df_base = df_reg[df_reg['index_fund_flag'].astype(str).str.contains('B', case=False, regex=True)]
df_pure = df_reg[df_reg['index_fund_flag'].astype(str).str.contains('D', case=False, regex=True)]
df_non_index = df_reg[~df_reg['index_fund_flag'].astype(str).str.contains('D|B|E', case=False, regex=True)]

In [10]:
index = regression(df_index)
enhanced = regression(df_enhanced)
base = regression(df_base)
pure = regression(df_pure)
non_index = regression(df_non_index)
panelC = pd.DataFrame({'All index funds': index.mean(), 'Enhanced': enhanced.mean(), 'Base': base.mean(), 
              'Pure': pure.mean(), 'All non-index funds': non_index.mean()}).T
panelC

,Mkt-RF,SMB,HML,MOM,CMA,RMW,flow
All index funds,1.001094,0.119048,0.035691,0.003202,0.040796,0.060536,0.017879
Enhanced,0.614237,0.141505,-0.483040,0.023068,0.385493,-0.304316,0.002768
Base,1.039379,0.249752,0.079521,0.002755,-0.012171,0.086793,0.018714
Pure,0.993969,0.056680,0.015089,0.004320,0.059489,0.041154,0.023412
All non-index funds,0.983937,0.228301,0.016520,0.015960,-0.042796,0.033508,0.016748
